In [200]:
#import re
import sys, codecs, optparse, glob, os
import string
import random
import itertools as it

### Attention à la gestion des points des parenthèses et des crochets
mettre *echapperParCro* à :
- *True* pour utiliser les parenthèses et les crochets dans les chaines
- *False* pour que les caractères en question soient considérés comme des éléments de **regex**

Pour l'instant, les points sont toujours échappés.

In [201]:
debug=False
echapperParCro=False

In [202]:
dossier="/Github/eGloss"
enonce="Test"
theme="CH-Separations-Mots"
numeroterQuestions=True
adaptatifPenalite="0.010000"
nbHints=24
dernierIndiceNote=95.0
msgPlusDeMots=u"Il y a plus de mots"
msgMoinsDeMots=u"Il y a moins de mots"
def nbIndiceNotes(nb):
    if nb==1: 
        return ["%50%"]
    elif nb==2:
        return ["","%50%"]
    else:
        return [""]+["%"+str(int((i+2)*dernierIndiceNote/nb))+"%" for i in range(nb-1)]   

In [203]:
if debug:
    for i in range(20):
        print i,nbIndiceNotes(i) 

nbIndiceNotes={
    1:["%50%"],
    2:["","%50%"],
    3:["","%40%","%75%"],
    4:["","%25%","%50%","%75%"],
    5:["","%25%","%40%", "%60%","%75%"],
    6:["","%20%","%35%","%50%", "%60%","%75%"],
    7:["","%15","%25%","%35%","%50%", "%60%","%75%"],
    8:["","%15","%25%","%35%","%45%","%55%", "%65%","%75%"],
    9:["","%5%","%15","%25%","%35%","%45%","%55%", "%65%","%75%"],
    10:["","%8%","%16%","%24%","%32%","%40%","%48%", "%56%","%64%","%75%"],
}

In [204]:
from os.path import expanduser
home = expanduser("~")
repertoire=home+dossier
base=repertoire+"/"

In [205]:
from ToolsMoodleXML2 import *
import yaml

In [206]:
yamlFichier=base+theme+"-"+enonce+".yaml"
csvFichier=base+theme+".txt"
boolEX=False
titreNoEX=False

In [207]:
with open(yamlFichier, 'r') as stream:
    configuration=yaml.load(stream)
    
with codecs.open(csvFichier, 'r', encoding="utf8") as file:
    donneesLignes=[l.strip() for l in file.readlines() if l.strip()!=""]
print donneesLignes

[u'ron jiyizkin ron ginxo geci winvxo ujuz jiyizvxon ron becug otmayakanoza ginhxoz ekyupenvu vangaz givxoz miznxo rizi otmavoloyo wanacag vopa uyumkizvxo']


In [208]:
def sousListes(liste, longueur):
    for i in range(0, len(liste), longueur):
        yield liste[i:i+longueur]
        
def sousSuites(liste,longueur):
    for i in range(0, len(liste)-longueur+1):
        yield liste[i:i+longueur]
    

In [209]:
def choixChunk(mots):
    choix=4
    longueur=len(mots)
    if longueur<=10:
        choix=3
    elif longueur<=20:
        choix=4
    elif longueur<=30:
        choix=5
    else:
        choix=6
    return choix

def choixChunk(mots):
    longueur=len(mots)
    choix=max(longueur/4,1)
    return choix

def choixPenaltyStages(question):
    longueur=len(question.split(" "))
    stages=longueur*2
    penalite=str(round(25.0/stages*0.01,4))
    return (penalite,stages)


def motsRegex(mots):
    indiceNotes=nbIndiceNotes(len(mots))
    pRegex=[]
    debutMots=u""
    nbMots=0
    chunkSize=choixChunk(mots)
    for chunk in sousListes(mots,chunkSize):
        lChunk=len(chunk)
        cRegex=[]
        for l in range(lChunk):
#            print l
            lRegex=u"".join([ur"(?!.*\b%s\b.*)"%(" ".join(s)) for s in sousSuites(chunk,l+1)])
#            print lRegex
            note=indiceNotes[l+nbMots]
            #
            # changer le texte du feedback => différent pour les débuts vides
            #
            lStrRegex=u"%s^%s(%s"%(note,debutMots,lRegex)+u').*#<span style="color:blue">%s</span><br/>%d correct(s) à la suite parmi les %d mots suivants.'%(debutMots.strip(),l,lChunk)
            print lStrRegex
            cRegex.append(lStrRegex)
        debutMots+=" ".join(chunk)+" "
        debutMots2=debutMots
#        cRegex.append(u"~%%10%%^(?!%s).*#Début : %d, Suite vous avez identifié les %d mots suivants, mais pas au bon endroit."%(debutMots,nbMots,lSetMots))
        pRegex.extend(cRegex)
        nbMots+=lChunk
    return pRegex

In [210]:
pRegex=motsRegex(u"ron jiyizkin ron ginxo geci winvxo ujuz jiyizvxon ron becug otmayakanoza ginhxoz ekyupenvu vangaz givxoz miznxo rizi otmavoloyo wanacag vopa uyumkizvxo".split(" "))
pRegex

^((?!.*\bron\b.*)(?!.*\bjiyizkin\b.*)(?!.*\bron\b.*)(?!.*\bginxo\b.*)(?!.*\bgeci\b.*)).*#<span style="color:blue"></span><br/>0 correct(s) à la suite parmi les 5 mots suivants.
%9%^((?!.*\bron jiyizkin\b.*)(?!.*\bjiyizkin ron\b.*)(?!.*\bron ginxo\b.*)(?!.*\bginxo geci\b.*)).*#<span style="color:blue"></span><br/>1 correct(s) à la suite parmi les 5 mots suivants.
%13%^((?!.*\bron jiyizkin ron\b.*)(?!.*\bjiyizkin ron ginxo\b.*)(?!.*\bron ginxo geci\b.*)).*#<span style="color:blue"></span><br/>2 correct(s) à la suite parmi les 5 mots suivants.
%18%^((?!.*\bron jiyizkin ron ginxo\b.*)(?!.*\bjiyizkin ron ginxo geci\b.*)).*#<span style="color:blue"></span><br/>3 correct(s) à la suite parmi les 5 mots suivants.
%22%^((?!.*\bron jiyizkin ron ginxo geci\b.*)).*#<span style="color:blue"></span><br/>4 correct(s) à la suite parmi les 5 mots suivants.
%27%^ron jiyizkin ron ginxo geci ((?!.*\bwinvxo\b.*)(?!.*\bujuz\b.*)(?!.*\bjiyizvxon\b.*)(?!.*\bron\b.*)(?!.*\bbecug\b.*)).*#<span style="color:blue"

[u'^((?!.*\\bron\\b.*)(?!.*\\bjiyizkin\\b.*)(?!.*\\bron\\b.*)(?!.*\\bginxo\\b.*)(?!.*\\bgeci\\b.*)).*#<span style="color:blue"></span><br/>0 correct(s) \xe0 la suite parmi les 5 mots suivants.',
 u'%9%^((?!.*\\bron jiyizkin\\b.*)(?!.*\\bjiyizkin ron\\b.*)(?!.*\\bron ginxo\\b.*)(?!.*\\bginxo geci\\b.*)).*#<span style="color:blue"></span><br/>1 correct(s) \xe0 la suite parmi les 5 mots suivants.',
 u'%13%^((?!.*\\bron jiyizkin ron\\b.*)(?!.*\\bjiyizkin ron ginxo\\b.*)(?!.*\\bron ginxo geci\\b.*)).*#<span style="color:blue"></span><br/>2 correct(s) \xe0 la suite parmi les 5 mots suivants.',
 u'%18%^((?!.*\\bron jiyizkin ron ginxo\\b.*)(?!.*\\bjiyizkin ron ginxo geci\\b.*)).*#<span style="color:blue"></span><br/>3 correct(s) \xe0 la suite parmi les 5 mots suivants.',
 u'%22%^((?!.*\\bron jiyizkin ron ginxo geci\\b.*)).*#<span style="color:blue"></span><br/>4 correct(s) \xe0 la suite parmi les 5 mots suivants.',
 u'%27%^ron jiyizkin ron ginxo geci ((?!.*\\bwinvxo\\b.*)(?!.*\\bujuz\\b.*)(?!.

In [211]:
print u"".join(pRegex)

^((?!.*\bron\b.*)(?!.*\bjiyizkin\b.*)(?!.*\bron\b.*)(?!.*\bginxo\b.*)(?!.*\bgeci\b.*)).*#<span style="color:blue"></span><br/>0 correct(s) à la suite parmi les 5 mots suivants.%9%^((?!.*\bron jiyizkin\b.*)(?!.*\bjiyizkin ron\b.*)(?!.*\bron ginxo\b.*)(?!.*\bginxo geci\b.*)).*#<span style="color:blue"></span><br/>1 correct(s) à la suite parmi les 5 mots suivants.%13%^((?!.*\bron jiyizkin ron\b.*)(?!.*\bjiyizkin ron ginxo\b.*)(?!.*\bron ginxo geci\b.*)).*#<span style="color:blue"></span><br/>2 correct(s) à la suite parmi les 5 mots suivants.%18%^((?!.*\bron jiyizkin ron ginxo\b.*)(?!.*\bjiyizkin ron ginxo geci\b.*)).*#<span style="color:blue"></span><br/>3 correct(s) à la suite parmi les 5 mots suivants.%22%^((?!.*\bron jiyizkin ron ginxo geci\b.*)).*#<span style="color:blue"></span><br/>4 correct(s) à la suite parmi les 5 mots suivants.%27%^ron jiyizkin ron ginxo geci ((?!.*\bwinvxo\b.*)(?!.*\bujuz\b.*)(?!.*\bjiyizvxon\b.*)(?!.*\bron\b.*)(?!.*\bbecug\b.*)).*#<span style="color:blue">ron 

In [212]:
def echapperCorrection(correction):
    result=correction
    print "original",result
    subPoints = re.compile(ur"(\.[^*+])")
    result=subPoints.sub(ur"\\\1",result)
    m=re.match(ur"^(--\.\*\()(&&.*)(\)\.\*)$",result)
    if m:
        (prefixe, corps, suffixe)=m.groups()
        subParentheses = re.compile(ur"\(([^)]*)\)")
        corps=subParentheses.sub(ur"\\(\1\\)",corps)
        subCrochets = re.compile(ur"\[([^]]*)\]")
        corps=subCrochets.sub(ur"\\[\1\\]",corps)
        result=prefixe+corps+suffixe
    print "échappé",result
    return result    

def modifierCorrection(correction):
    result=correction
    print "original",result
    subPoints = re.compile(ur"(\.[^*+])")
    result=subPoints.sub(ur"\\\1",result)
    print "\tsubPoints", result
    m=re.match(ur"^(--\.\*\()(&&.*)(\)\.\*)$",result)
    if m:
        (prefixe, corps, suffixe)=m.groups()
        if echapperParCro:
            subParentheses = re.compile(ur"\(([^)]*)\)")
            corps=subParentheses.sub(ur"\\(\1\\)",corps)
            print "\tsubParentheses",corps
            subCrochets = re.compile(ur"\[([^]]*)\]")
            corps=subCrochets.sub(ur"\\[\1\\]",corps)
            print "\tsubCrochets",corps
        unites=filter(None,corps.split("&&"))
        print "\tunités",unites
        corps=".*".join(unites)
        result=prefixe+corps+suffixe
    print "\téchappé",result
    return result    


def separerMots(elements,titre):
    result=[]
    elements = filter(None, elements)
    mots=elements[0].strip().split(" ")
    nbMots=len(mots)
    solution=u"=%s#Bravo ! C'est la bonne phrase."%(elements[0])
    corrections=[]
    corrections=motsRegex(mots)
    nbElements=len(elements[1:])
#    if not nbElements in nbIndiceNotes: 
#        print "Ajouter une répartition des notes pour %d indices"%nbElements
#        print titre
#        print elements[1:]
    for n,element in enumerate(elements[1:]):
        (reponseSpec,feedbackSpec)=element.split("#")
        reponseSpec=modifierCorrection(reponseSpec)
        element="#".join([reponseSpec,feedbackSpec])
        corrections.append(nbIndiceNotes(nbElements)[n]+element)
#    print nbMots
    minMots="^"+"([^ ]+ +)?"*(nbMots-4)+"[^ ]+$#%s"%msgPlusDeMots
    maxMots="^"+"([^ ]+ +)"*(nbMots+2)+"([^ ]+ +)*[^ ]+$#%s"%msgMoinsDeMots
#    print "~".join(elements)
    #
    # Supprimer les minMots et les maxMots
    #
    # result.extend([solution,minMots,maxMots])
    result.extend([solution])
    result.extend(corrections)
    return result

In [213]:
categorie=configuration[u"Catégorie"]
print categorie
clozeQuestions=XMLSeries(categorie)

Découpage des phrases/Charabia/Cloze


In [214]:
xmlOutput=MoodleXML()

In [215]:
def makeCharabiaSepEx(donneesLignes,nCharabia):
    consigne="<br/>".join(configuration[u"Consigne"])
    separationExercices=[]
    clozeQuestions=XMLSeries(categorie.replace(u"Charabia/",u"Charabia%02d/"%nCharabia))
    
    for numQuestion,question in enumerate(donneesLignes):
        lQuestion=len(question.split(" "))
        questionLignes=[]
        titre="Phrase %03d"%(numQuestion+1)
        questionLignes.append(consigne)
        questionLignes.append(question.replace(" ","")+" (Phrase %03d)"%(numQuestion+1))
        questionLignes.append(u"{%d:RX:%s}"%(lQuestion,"~".join(separerMots([question],titre))))
        textQuestion=u"<br/>".join(questionLignes)
        #
        # Changer le nombre d'essais en fonction de la longueur de la phrase
        # Changer la pénalité en fonction du nombre d'essais
        #
        (adaptatifPenalite,nbHints)=choixPenaltyStages(question)
        separationExercices.append(ClozeExercice(titre,textQuestion,penalty=adaptatifPenalite,nbHints=nbHints))

    for exercice in separationExercices:
        clozeQuestions.addExercice(exercice)
        
    xmlOutput.addQuiz(clozeQuestions.getSeries())

In [216]:
def getDonnees(nCharabia):
    filtrePonct=u":,;()\t-’´'`…–‚«»"+'"' # ici l'espace n'est pas filtré
    charabiaLignes={}
    rep="/Users/gilles/ownCloud/Cours/Bordeaux/L3-Morphologie3/Grimm/"
    stops=re.escape(u".?!")+"\n"
    f=rep+"Extraits-%02d-Lot3-brut.text"%(nCharabia)
    texte=[]
    with codecs.open(f,"r",encoding="utf8") as inFile:
        lignes="\n".join([l.strip() for l in inFile.readlines() if l.strip()!="" and l.strip()[0]!="#"])
        texte.extend([re.sub(ur"[%s]"%re.escape(filtrePonct),"", l).strip() for l in re.split(u"[%s]"%stops,lignes) if l!=""])
    texte=[t for t in texte if t!=""]
    return texte

In [217]:
for n in range(1,12):
    makeCharabiaSepEx(getDonnees(n),n)

^((?!.*\bpeq\b.*)(?!.*\bcaxdi\b.*)(?!.*\bcaxdi\b.*)(?!.*\btaq\b.*)(?!.*\bwozix\b.*)(?!.*\byirizi\b.*)).*#<span style="color:blue"></span><br/>0 correct(s) à la suite parmi les 6 mots suivants.
%7%^((?!.*\bpeq caxdi\b.*)(?!.*\bcaxdi caxdi\b.*)(?!.*\bcaxdi taq\b.*)(?!.*\btaq wozix\b.*)(?!.*\bwozix yirizi\b.*)).*#<span style="color:blue"></span><br/>1 correct(s) à la suite parmi les 6 mots suivants.
%10%^((?!.*\bpeq caxdi caxdi\b.*)(?!.*\bcaxdi caxdi taq\b.*)(?!.*\bcaxdi taq wozix\b.*)(?!.*\btaq wozix yirizi\b.*)).*#<span style="color:blue"></span><br/>2 correct(s) à la suite parmi les 6 mots suivants.
%14%^((?!.*\bpeq caxdi caxdi taq\b.*)(?!.*\bcaxdi caxdi taq wozix\b.*)(?!.*\bcaxdi taq wozix yirizi\b.*)).*#<span style="color:blue"></span><br/>3 correct(s) à la suite parmi les 6 mots suivants.
%18%^((?!.*\bpeq caxdi caxdi taq wozix\b.*)(?!.*\bcaxdi caxdi taq wozix yirizi\b.*)).*#<span style="color:blue"></span><br/>4 correct(s) à la suite parmi les 6 mots suivants.
%21%^((?!.*\bpeq caxdi

%39%^zqexxoxdix ptezhi ix zahhiq ziq raq ((?!.*\bwta wcus ah sexdix\b.*)(?!.*\bwcus ah sexdix jyir\b.*)(?!.*\bah sexdix jyir uzi\b.*)).*#<span style="color:blue">zqexxoxdix ptezhi ix zahhiq ziq raq</span><br/>3 correct(s) à la suite parmi les 6 mots suivants.
%43%^zqexxoxdix ptezhi ix zahhiq ziq raq ((?!.*\bwta wcus ah sexdix jyir\b.*)(?!.*\bwcus ah sexdix jyir uzi\b.*)).*#<span style="color:blue">zqexxoxdix ptezhi ix zahhiq ziq raq</span><br/>4 correct(s) à la suite parmi les 6 mots suivants.
%47%^zqexxoxdix ptezhi ix zahhiq ziq raq ((?!.*\bwta wcus ah sexdix jyir uzi\b.*)).*#<span style="color:blue">zqexxoxdix ptezhi ix zahhiq ziq raq</span><br/>5 correct(s) à la suite parmi les 6 mots suivants.
%51%^zqexxoxdix ptezhi ix zahhiq ziq raq wta wcus ah sexdix jyir uzi ((?!.*\bap\b.*)(?!.*\bwod\b.*)(?!.*\bwiyr\b.*)(?!.*\bap\b.*)(?!.*\bdytizi\b.*)(?!.*\bed\b.*)).*#<span style="color:blue">zqexxoxdix ptezhi ix zahhiq ziq raq wta wcus ah sexdix jyir uzi</span><br/>0 correct(s) à la suite parm

%66%^dezzad ceqyoy wadzi vodix ñraz iq ((?!.*\bzih\b.*)(?!.*\bzed\b.*)).*#<span style="color:blue">dezzad ceqyoy wadzi vodix ñraz iq</span><br/>0 correct(s) à la suite parmi les 2 mots suivants.
%76%^dezzad ceqyoy wadzi vodix ñraz iq ((?!.*\bzih zed\b.*)).*#<span style="color:blue">dezzad ceqyoy wadzi vodix ñraz iq</span><br/>1 correct(s) à la suite parmi les 2 mots suivants.
%85%^dezzad ceqyoy wadzi vodix ñraz iq zih zed ((?!.*\bzu\b.*)(?!.*\bjiwhoyyiq\b.*)).*#<span style="color:blue">dezzad ceqyoy wadzi vodix ñraz iq zih zed</span><br/>0 correct(s) à la suite parmi les 2 mots suivants.
%95%^dezzad ceqyoy wadzi vodix ñraz iq zih zed ((?!.*\bzu jiwhoyyiq\b.*)).*#<span style="color:blue">dezzad ceqyoy wadzi vodix ñraz iq zih zed</span><br/>1 correct(s) à la suite parmi les 2 mots suivants.
^((?!.*\btid\b.*)(?!.*\bwvoxziq\b.*)).*#<span style="color:blue"></span><br/>0 correct(s) à la suite parmi les 2 mots suivants.
%21%^((?!.*\btid wvoxziq\b.*)).*#<span style="color:blue"></span><br/>1 

%54%^oxzi o ñuwih waz pyuiqxi vta rtiddix ed wer sessix whez ciz evytephih ñtaxz peq ah ((?!.*\bdori zqixdix ix\b.*)(?!.*\bzqixdix ix teqipodix\b.*)(?!.*\bix teqipodix ed\b.*)(?!.*\bteqipodix ed vodix\b.*)(?!.*\bed vodix waz\b.*)(?!.*\bvodix waz ciz\b.*)).*#<span style="color:blue">oxzi o ñuwih waz pyuiqxi vta rtiddix ed wer sessix whez ciz evytephih ñtaxz peq ah</span><br/>2 correct(s) à la suite parmi les 8 mots suivants.
%57%^oxzi o ñuwih waz pyuiqxi vta rtiddix ed wer sessix whez ciz evytephih ñtaxz peq ah ((?!.*\bdori zqixdix ix teqipodix\b.*)(?!.*\bzqixdix ix teqipodix ed\b.*)(?!.*\bix teqipodix ed vodix\b.*)(?!.*\bteqipodix ed vodix waz\b.*)(?!.*\bed vodix waz ciz\b.*)).*#<span style="color:blue">oxzi o ñuwih waz pyuiqxi vta rtiddix ed wer sessix whez ciz evytephih ñtaxz peq ah</span><br/>3 correct(s) à la suite parmi les 8 mots suivants.
%60%^oxzi o ñuwih waz pyuiqxi vta rtiddix ed wer sessix whez ciz evytephih ñtaxz peq ah ((?!.*\bdori zqixdix ix teqipodix ed\b.*)(?!.*\bzqixdi

%67%^ron jiyizkin ron ginxo geci winvxo ujuz jiyizvxon ron becug ((?!.*\botmayakanoza ginhxoz ekyupenvu vangaz givxoz\b.*)).*#<span style="color:blue">ron jiyizkin ron ginxo geci winvxo ujuz jiyizvxon ron becug</span><br/>4 correct(s) à la suite parmi les 5 mots suivants.
%72%^ron jiyizkin ron ginxo geci winvxo ujuz jiyizvxon ron becug otmayakanoza ginhxoz ekyupenvu vangaz givxoz ((?!.*\bmiznxo\b.*)(?!.*\brizi\b.*)(?!.*\botmavoloyo\b.*)(?!.*\bwanacag\b.*)(?!.*\bvopa\b.*)).*#<span style="color:blue">ron jiyizkin ron ginxo geci winvxo ujuz jiyizvxon ron becug otmayakanoza ginhxoz ekyupenvu vangaz givxoz</span><br/>0 correct(s) à la suite parmi les 5 mots suivants.
%76%^ron jiyizkin ron ginxo geci winvxo ujuz jiyizvxon ron becug otmayakanoza ginhxoz ekyupenvu vangaz givxoz ((?!.*\bmiznxo rizi\b.*)(?!.*\brizi otmavoloyo\b.*)(?!.*\botmavoloyo wanacag\b.*)(?!.*\bwanacag vopa\b.*)).*#<span style="color:blue">ron jiyizkin ron ginxo geci winvxo ujuz jiyizvxon ron becug otmayakanoza ginhxoz ekyu

...

%47%^uyu zoiq joi yañymojby fe lohhjw foi qzue hoixjwi ((?!.*\byahim uxiz fiz jbyañmi rapih jowkw mobyw viyz ov\b.*)).*#<span style="color:blue">uyu zoiq joi yañymojby fe lohhjw foi qzue hoixjwi</span><br/>8 correct(s) à la suite parmi les 9 mots suivants.
%50%^uyu zoiq joi yañymojby fe lohhjw foi qzue hoixjwi yahim uxiz fiz jbyañmi rapih jowkw mobyw viyz ov ((?!.*\bmijw\b.*)(?!.*\bemf\b.*)(?!.*\bjompw\b.*)(?!.*\bmobyw\b.*)(?!.*\bviyz\b.*)(?!.*\bfoi\b.*)(?!.*\bguwki\b.*)(?!.*\byuw\b.*)(?!.*\boym\b.*)).*#<span style="color:blue">uyu zoiq joi yañymojby fe lohhjw foi qzue hoixjwi yahim uxiz fiz jbyañmi rapih jowkw mobyw viyz ov</span><br/>0 correct(s) à la suite parmi les 9 mots suivants.
%52%^uyu zoiq joi yañymojby fe lohhjw foi qzue hoixjwi yahim uxiz fiz jbyañmi rapih jowkw mobyw viyz ov ((?!.*\bmijw emf\b.*)(?!.*\bemf jompw\b.*)(?!.*\bjompw mobyw\b.*)(?!.*\bmobyw viyz\b.*)(?!.*\bviyz foi\b.*)(?!.*\bfoi guwki\b.*)(?!.*\bguwki yuw\b.*)(?!.*\byuw oym\b.*)).*#<span style="color:blue">uyu 

%59%^hfuwturb mterb jten emow temmp e mtelter e ztenexphtewptim jtuwhuj ltatsepipp tewble ((?!.*\bipp qummupp qaciniwroe\b.*)(?!.*\bqummupp qaciniwroe e\b.*)(?!.*\bqaciniwroe e patfsumb\b.*)(?!.*\be patfsumb jummupp\b.*)).*#<span style="color:blue">hfuwturb mterb jten emow temmp e mtelter e ztenexphtewptim jtuwhuj ltatsepipp tewble</span><br/>2 correct(s) à la suite parmi les 6 mots suivants.
%63%^hfuwturb mterb jten emow temmp e mtelter e ztenexphtewptim jtuwhuj ltatsepipp tewble ((?!.*\bipp qummupp qaciniwroe e\b.*)(?!.*\bqummupp qaciniwroe e patfsumb\b.*)(?!.*\bqaciniwroe e patfsumb jummupp\b.*)).*#<span style="color:blue">hfuwturb mterb jten emow temmp e mtelter e ztenexphtewptim jtuwhuj ltatsepipp tewble</span><br/>3 correct(s) à la suite parmi les 6 mots suivants.
%67%^hfuwturb mterb jten emow temmp e mtelter e ztenexphtewptim jtuwhuj ltatsepipp tewble ((?!.*\bipp qummupp qaciniwroe e patfsumb\b.*)(?!.*\bqummupp qaciniwroe e patfsumb jummupp\b.*)).*#<span style="color:blue">hfuwt

%10%^((?!.*\be qontembzo\b.*)(?!.*\bqontembzo jihp\b.*)(?!.*\bjihp oh\b.*)).*#<span style="color:blue"></span><br/>1 correct(s) à la suite parmi les 4 mots suivants.
%15%^((?!.*\be qontembzo jihp\b.*)(?!.*\bqontembzo jihp oh\b.*)).*#<span style="color:blue"></span><br/>2 correct(s) à la suite parmi les 4 mots suivants.
%21%^((?!.*\be qontembzo jihp oh\b.*)).*#<span style="color:blue"></span><br/>3 correct(s) à la suite parmi les 4 mots suivants.
%26%^e qontembzo jihp oh ((?!.*\bjummtutu\b.*)(?!.*\bhfuwitxiwpp\b.*)(?!.*\befp\b.*)(?!.*\bwirximpe\b.*)).*#<span style="color:blue">e qontembzo jihp oh</span><br/>0 correct(s) à la suite parmi les 4 mots suivants.
%31%^e qontembzo jihp oh ((?!.*\bjummtutu hfuwitxiwpp\b.*)(?!.*\bhfuwitxiwpp efp\b.*)(?!.*\befp wirximpe\b.*)).*#<span style="color:blue">e qontembzo jihp oh</span><br/>1 correct(s) à la suite parmi les 4 mots suivants.
%36%^e qontembzo jihp oh ((?!.*\bjummtutu hfuwitxiwpp efp\b.*)(?!.*\bhfuwitxiwpp efp wirximpe\b.*)).*#<span style="

%51%^joqin sejacocitqu lumtucupp e ((?!.*\bpunujlu jorxurqoruq\b.*)).*#<span style="color:blue">joqin sejacocitqu lumtucupp e</span><br/>1 correct(s) à la suite parmi les 2 mots suivants.
%60%^joqin sejacocitqu lumtucupp e punujlu jorxurqoruq ((?!.*\bumeqexp\b.*)(?!.*\be\b.*)).*#<span style="color:blue">joqin sejacocitqu lumtucupp e punujlu jorxurqoruq</span><br/>0 correct(s) à la suite parmi les 2 mots suivants.
%69%^joqin sejacocitqu lumtucupp e punujlu jorxurqoruq ((?!.*\bumeqexp e\b.*)).*#<span style="color:blue">joqin sejacocitqu lumtucupp e punujlu jorxurqoruq</span><br/>1 correct(s) à la suite parmi les 2 mots suivants.
%77%^joqin sejacocitqu lumtucupp e punujlu jorxurqoruq umeqexp e ((?!.*\bhfeye\b.*)(?!.*\bef\b.*)).*#<span style="color:blue">joqin sejacocitqu lumtucupp e punujlu jorxurqoruq umeqexp e</span><br/>0 correct(s) à la suite parmi les 2 mots suivants.
%86%^joqin sejacocitqu lumtucupp e punujlu jorxurqoruq umeqexp e ((?!.*\bhfeye ef\b.*)).*#<span style="color:blue">jo

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [218]:
xmlFichier=yamlFichier.split(".")[0]+".xml"

In [219]:
with codecs.open(xmlFichier, 'w',encoding="utf8") as output:
#    print [questions.getClozes()]
    output.write(xmlOutput.getXML())

In [198]:
print xmlOutput.getXML()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [199]:
re.sub(ur"[%s]"%re.escape(filtrePonct),"", u"dqmslfjk,qsdflkjqs")
#print ur"[%s]"%re.escape(filtrePonct)

NameError: name 'filtrePonct' is not defined